In [ ]:
from plotly.io import show
from sklearn.model_selection import train_test_split
from skfolio import Population
from skfolio.optimization import EqualWeighted, MaximumDiversification, Random
from skfolio.preprocessing import prices_to_returns
from openbb import obb
from itertools import chain, combinations

In [ ]:
sectors = [
    "SPY",
    "QYLD",
    "XYLD",
    "ALV.DE",
    "CDR.WA",
]

In [ ]:
def powerset(iterable):
    """
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    xs = list(iterable)
    # note we return an iterator rather than a list
    return chain.from_iterable(combinations(xs, n) for n in range(2, len(xs) + 1))

In [ ]:
df = obb.equity.price.historical(
    sectors, start_date="2015-01-01", provider="yfinance"
).to_df()
pivoted = df.pivot(columns="symbol", values="close").dropna()
pivoted

In [ ]:
X = prices_to_returns(pivoted)

X_train, X_test = train_test_split(X, test_size=0.2, shuffle=False)
X

In [ ]:
model = MaximumDiversification()
model.fit(X_train)
ptf_model_train = model.predict(X_train)

bench = EqualWeighted()
bench.fit(X_train)
ptf_bench_train = bench.predict(X_train)

random = Random()
random.fit(X_train)
ptf_random_train = random.predict(X_train)

print(f"Maximum Diversification: {ptf_model_train.diversification:0.2f}")
print(f"Equal Weighted model: {ptf_bench_train.diversification:0.2f}")
print(f"Random Weighted model: {ptf_random_train.diversification:0.2f}")

In [ ]:
ptf_model_test = model.predict(X_test)
ptf_bench_test = bench.predict(X_test)
ptf_random_test = random.predict(X_test)
population = Population([ptf_model_test, ptf_bench_test, ptf_random_test])

In [ ]:
population.plot_composition()

In [ ]:
population.plot_cumulative_returns()

In [ ]:
population.summary()